# Retrieval

Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow. 

Let's get our vectorDB from before.

## Vectorstore retrieval


In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
#!pip install lark

### Similarity Search

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [4]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [5]:
print(vectordb._collection.count())

1531


In [7]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [8]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [9]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [10]:
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [11]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

### Addressing Diversity: Maximum marginal relevance

Last class we introduced one problem: how to enforce diversity in the search results.
 
`Maximum marginal relevance` strives to achieve both relevance to the query *and diversity* among the results.

In [12]:
question = "Wo wird überall Risikobeurteilung durch unabhängige externe Stellen erwähnt?"
docs_ss = vectordb.similarity_search(question,k=3)

In [14]:
docs_ss[0].page_content[:1000]

'eingesetzt werden sollen und bei denen Genauigkeit, Zuverlässigkeit und Transparenz \nbesonders wichtig sind, als hochriskant einzustufen, um nachteilige Auswirkungen zu \nvermeiden, das Vertrauen der Öffentlichkeit zu erhalten und die Rechenschaftspflicht und \neinen wirksamen Rechtsschutz zu gewährleisten.'

In [15]:
docs_ss[1].page_content[:1000]

'eingesetzt werden sollen und bei denen Genauigkeit, Zuverlässigkeit und Transparenz \nbesonders wichtig sind, als hochriskant einzustufen, um nachteilige Auswirkungen zu \nvermeiden, das Vertrauen der Öffentlichkeit zu erhalten und die Rechenschaftspflicht und \neinen wirksamen Rechtsschutz zu gewährleisten.'

Note the difference in results with `MMR`.

In [16]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [17]:
docs_mmr[0].page_content[:100]

'eingesetzt werden sollen und bei denen Genauigkeit, Zuverlässigkeit und Transparenz \nbesonders wicht'

In [18]:
docs_mmr[1].page_content[:100]

'systemische Risiken, die sich aus potenziellen Fehlanwendungen ergeben, zu bewerten. \nDiese Anbieter'

### Addressing Specificity: working with metadata

In last lecture, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on `metadata`.

`metadata` provides context for each embedded chunk.

In [23]:
question = "Wie sind in der DSGVO technische und organisatorische Maßnahmen definiert?"

In [26]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"pdfs/AI-Act.pdf"}
)

In [28]:
for d in docs:
    print(d.metadata)
    print(d.page_content[:100])

{'page': 22, 'source': 'pdfs/DSGVO.pdf'}
02016R0679 — DE — 04.05.2016 — 000.003 — 23  
(2) Sofern dies in einem angemessenen Verhältnis zu de
{'page': 395, 'source': 'pdfs/AI-Act.pdf'}
396/460 RR\A9-0188/2023DE.docx
DEb) Grad der Verantwortung des Organs, der Einrichtung oder der sons
{'page': 395, 'source': 'pdfs/AI-Act.pdf'}
396/460 RR\A9-0188/2023DE.docx
DEb) Grad der Verantwortung des Organs, der Einrichtung oder der sons


### Addressing Specificity: working with metadata using self-query retriever

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
 
1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [29]:
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [30]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The document chunk is from, should be one of `pdfs/AI-Act.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the AI-Act document",
        type="integer",
    ),
]

**Note:** The default model for `OpenAI` ("from langchain.llms import OpenAI") is `text-davinci-003`. Due to the deprication of OpenAI's model `text-davinci-003` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [31]:
document_content_description = "AI-Act"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [36]:
question = "Wo werden überall im AI-Act Katzen erwähnt?"

**You will receive a warning** about predict_and_parse being deprecated the first time you executing the next line. This can be safely ignored.

In [37]:
docs = retriever.invoke(question)

In [38]:
for d in docs:
    print(d.metadata)
    print(d.page_content[:200])

{'page': 117, 'source': 'pdfs/AI-Act.pdf'}
bestimmten Kategorien zuordnen können. Solche spezifischen Kategorien können 
Aspekte wie etwa Geschlecht, Alter, Haarfarbe, Augenfarbe, Tätowierungen, 
persönliche Merkmale, ethnische Herkunft sowie 
{'page': 117, 'source': 'pdfs/AI-Act.pdf'}
bestimmten Kategorien zuordnen können. Solche spezifischen Kategorien können 
Aspekte wie etwa Geschlecht, Alter, Haarfarbe, Augenfarbe, Tätowierungen, 
persönliche Merkmale, ethnische Herkunft sowie 
{'page': 89, 'source': 'pdfs/AI-Act.pdf'}
KI-System betroffen sein könnten, unabhängige Sachverständige und Organisationen 
der Zivilgesellschaft, in die Durchführung solcher Folgenabschätzungen und die 
Gestaltung von Maßnahmen, die im Falle
{'page': 89, 'source': 'pdfs/AI-Act.pdf'}
KI-System betroffen sein könnten, unabhängige Sachverständige und Organisationen 
der Zivilgesellschaft, in die Durchführung solcher Folgenabschätzungen und die 
Gestaltung von Maßnahmen, die im Falle


### Additional tricks: compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text. 

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. 

In [39]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [40]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\nSource: {d.metadata}\n\n" + d.page_content for i, d in enumerate(docs)]))


In [41]:
# Wrap our vectorstore
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
compressor = LLMChainExtractor.from_llm(llm)

In [43]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [44]:
question = "Wo wird Datenschutz im AI-Act erwähnt?"
compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

Document 1:

Source: {'page': 68, 'source': 'pdfs/AI-Act.pdf'}

- Das Recht auf Privatsphäre und den Schutz personenbezogener Daten muss während des gesamten Lebenszyklus des KI-Systems sichergestellt sein.
- In dieser Hinsicht gelten die Grundsätze der Datenminimierung und des Datenschutzes durch Technikgestaltung und Voreinstellungen, wie sie im Datenschutzrecht der Union festgelegt sind, wenn personenbezogene Daten verarbeitet werden.
----------------------------------------------------------------------------------------------------
Document 2:

Source: {'page': 295, 'source': 'pdfs/AI-Act.pdf'}

- Anbieter von KI-Modellen mit allgemeinem Verwendungszweck mit systemischem Risiko 
- harmonisierten Norm 
- Praxisleitfäden 
- Einhaltung der in Absatz 1 des vorliegenden Artikels genannten Pflichten 
- harmonisierten europäischen Norm 
- Vermutung der Konformität 
- Anbieter von KI-Modellen mit allgemeinem Verwendungszweck mit systemischem Risiko, die einen genehmigten Praxisleitfaden n

## Combining various techniques

In [45]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [46]:
question = "Wo wird überall ein unabhängiger Sachverständiger im AI-Act erwähnt?"
compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

Document 1:

Source: {'page': 380, 'source': 'pdfs/AI-Act.pdf'}

- Die Kommission kann beschließen, unabhängige Sachverständige zu benennen, die in ihrem Namen Bewertungen durchführen, einschließlich aus dem gemäß Artikel 68 eingesetzten wissenschaftlichen Gremium.
- Die für diese Aufgabe benannten unabhängigen Sachverständigen erfüllen die in Artikel 68 Absatz 2 umrissenen Kriterien.
- Im Falle juristischer Personen, Gesellschaften oder – wenn der Anbieter keine Rechtspersönlichkeit besitzt – die Personen, die nach Gesetz oder ihrer Satzung zur Vertretung dieser Personen befugt sind, stellen den angeforderten Zugang im Namen des Anbieters des betreffenden KI-Modells mit allgemeinem Verwendungszweck zur Verfügung.
----------------------------------------------------------------------------------------------------
Document 2:

Source: {'page': 135, 'source': 'pdfs/AI-Act.pdf'}

- unabhängiger Sachverständiger
- AI-Act
- Fachwissen im KI-Bereich
- Aufgaben unparteiisch, objektiv und unte

## Other types of retrieval

It's worth noting that vectordb as not the only kind of tool to retrieve documents. 

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [47]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [48]:
# Load PDF
loader = PyPDFLoader("pdfs/AI-Act.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [49]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [50]:
question = "Wo wird überall KI-System im AI-Act erwähnt?"
docs_svm=svm_retriever.invoke(question)
docs_svm[0]

Document(page_content='Datenverarbeitung hinaus, indem Lern-, Schlussfolgerungs- und Modellierungsprozesse \nermöglicht werden. Die Bezeichnung „maschinenbasiert“ bezieht sich auf die Tatsache, \ndass KI-Systeme von Maschinen betrieben werden.  14/460 RR\\A9-0188/2023DE.docx\nDEDurch die Bezugnahme auf explizite oder implizite Ziele wird betont, dass KI-Systeme \ngemäß explizit festgelegten Zielen oder gemäß impliziten Zielen arbeiten können. Die \nZiele des KI-Systems können sich – unter bestimmten Umständen – von der \nZweckbestimmung des KI-Systems unterscheiden. Für die Zwecke dieser Verordnung \nsollten Umgebungen als Kontexte verstanden werden, in denen KI-Systeme betrieben \nwerden, während die von einem KI-System erzeugten Ausgaben verschiedene \nFunktionen von KI-Systemen widerspiegeln, darunter Vorhersagen, Inhalte, \nEmpfehlungen oder Entscheidungen. KI-Systeme sind mit verschiedenen Graden der \nAutonomie ausgestattet, was bedeutet, dass sie bis zu einem gewissen Grad unabh

In [51]:
question = "Wo wird überall KI-System im AI-Act erwähnt?"
docs_tfidf=tfidf_retriever.invoke(question)
docs_tfidf[0]

Document(page_content='auswirken, wenn das KI-System beispielsweise falsche Entscheidungen trifft oder falsche \noder verzerrte Ausgaben hervorbringt.\n(76) Die Cybersicherheit spielt eine entscheidende Rolle, wenn es darum geht, sicherzustellen, \ndass KI-Systeme widerstandsfähig gegenüber Versuchen böswilliger Dritter sind, unter \nAusnutzung der Schwachstellen der Systeme deren Verwendung, Verhalten, Leistung zu \nverändern oder ihre Sicherheitsmerkmale zu beeinträchtigen. Cyberangriffe auf KI-\nSysteme können KI-spezifische Ressourcen wie Trainingsdatensätze (z. B. \nDatenvergiftung) oder trainierte Modelle (z. B. feindliche Angriffe oder Inferenzangriffe \nauf Mitgliederdaten) nutzen oder Schwachstellen in den digitalen Ressourcen des KI-\nSystems oder der zugrunde liegenden IKT-Infrastruktur ausnutzen. Um ein den Risiken \nangemessenes Cybersicherheitsniveau zu gewährleisten, sollten die Anbieter von \nHochrisiko-KI-Systemen daher geeignete Maßnahmen, etwa Sicherheitskontrollen, 